In [1]:
import pandas as pd
import numpy as np
import datetime

%load_ext line_profiler
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
data_path = './data/'
!ls -lthr data/

total 2.8G
-rw-rw-r-- 1 sam sam 507M Feb 20 19:18 test.csv
-rw-rw-r-- 1 sam sam 2.0G Feb 20 19:18 train.csv
-rw-rw-r-- 1 sam sam 246M Feb 20 19:19 item_metadata.csv
-rw-rw-r-- 1 sam sam  51M Feb 20 19:25 submission_popular.csv


In [5]:
%%time
nrows = 10000
# nrows = None
train = pd.read_csv(data_path+'train.csv', nrows=nrows)

CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 15 ms


In [6]:
test = pd.read_csv(data_path+'test.csv', nrows=1000)
test.shape

(1000, 12)

In [7]:
# test_click_session_ids = test[test.impressions.notnull()].session_id.unique()

In [8]:
# ts = np.random.choice(test_click_session_ids, 1)
# test[test.session_id==ts[0]]

In [9]:
# train_click_ids = train[train.impressions.notnull()].session_id.unique()

In [10]:
# ts = np.random.choice(train_click_ids, 1)
# train[train.session_id==ts[0]]

In [11]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['reference']).isna().sum()

In [12]:
# train.groupby('session_id').apply(lambda x: x.iloc[-1]['impressions']).isna().sum()

### clip sessions off to last click

In [13]:
%%time
# get all rows upto the last clickout action (some rows has reference but it's not clickout action)
def up_to_last_click(grp):
    check = grp.action_type == 'clickout item'
    if check.sum() != 0:
        return grp.iloc[:np.argwhere(check)[-1][0]+1]
    else:
        return grp
        
train = train.groupby('session_id').apply(up_to_last_click).reset_index(drop=True)

/home/sam/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


CPU times: user 295 ms, sys: 4.46 ms, total: 299 ms
Wall time: 305 ms


In [17]:
%%time
# get utc timestamp
train['ts'] = train['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))

CPU times: user 3.47 ms, sys: 0 ns, total: 3.47 ms
Wall time: 3.33 ms


In [18]:
%%time
# only look at sessions with clickouts (for now)
# first filter out sessions that does not have a click-out
def check_clickout(grp):
    return 'clickout item' in grp['action_type'].unique()

clicked = train.groupby('session_id').apply(check_clickout)
click_session_ids = clicked[clicked].index
# filter
train = train[train.session_id.isin(click_session_ids)].reset_index(drop=True)

CPU times: user 67.6 ms, sys: 3.7 ms, total: 71.3 ms
Wall time: 70.6 ms


### meta info

In [26]:
%%time
meta = pd.read_csv(data_path+'item_metadata.csv', nrows=nrows)
meta['properties'] = meta['properties'].str.lower()

CPU times: user 30.1 ms, sys: 251 µs, total: 30.4 ms
Wall time: 30 ms


In [27]:
%%time
# checkout properties of items
def get_property_count(row):
    return len(row.split('|'))

item_p_ctn = meta['properties'].apply(get_property_count)
item_p_ctn.describe()

CPU times: user 22.6 ms, sys: 0 ns, total: 22.6 ms
Wall time: 22.1 ms


In [28]:
%%time
def get_property(row):
    return row.lower().split('|')
meta['ps'] = meta['properties'].apply(get_property)
# numer of properties
meta['nprop'] = meta.ps.str.len()
# star ratings
meta['star'] = meta.properties.str.extract('[\|](\d) star')


CPU times: user 37.3 ms, sys: 137 µs, total: 37.4 ms
Wall time: 36.8 ms


In [29]:
%%time
# create rating columns
ratings = ['good rating', 'satisfactory rating', 'excellent rating']
for r in ratings:
    meta[r.replace(' ', '_')] = meta.properties.str.findall(f'\|{r}').str.len()

CPU times: user 28.4 ms, sys: 4.05 ms, total: 32.4 ms
Wall time: 32.6 ms


In [ ]:
# total_ps = []
# for p in meta['ps'].values:
#     total_ps += p

In [ ]:
# total_ps

In [ ]:
# pd.value_counts(total_ps).iloc[:20].plot(kind='bar')

In [ ]:
meta.head()

In [ ]:
# meta.shape

In [ ]:
meta = (meta[['item_id', 'nprop', 'star', 'good_rating', 'satisfactory_rating', 'excellent_rating']]
        .set_index('item_id'))

### Create session features
---

In [ ]:
%%time
# find out if same customers have multiple sessions
s = train.drop_duplicates(subset=['user_id', 'session_id'])
s.session_id.duplicated().sum()
# seems like not

In [ ]:
# train.current_filters[:1000].unique()

In [89]:
%%time
def get_filters(x):
    if type(x) == str or type(x) == list:
        return x.split('|')
    else:
        return np.nan
def get_impressions(x):
    if type(x) == str:
        return x.split('|')
    else:
        return np.nan
    
    
train['filters'] = train.current_filters.str.split('|')
train['nfilters'] = train.filters.str.len()
train['imps_list'] = train.impressions.str.split('|')
nn_mask = train['imps_list'].notnull()
train.loc[nn_mask, 'imps_list'] = train.loc[nn_mask, 'imps_list'].apply(lambda x: [int(i) for i in x])
train['nimps'] = train.imps_list.str.len()

CPU times: user 9.86 ms, sys: 209 µs, total: 10.1 ms
Wall time: 9.37 ms


In [ ]:
train.nimps.describe()

In [ ]:
# no switch of devices during session
(train.groupby('session_id')['device'].nunique()!=1).sum()

In [ ]:
# differnet city during session
(train.groupby('session_id')['city'].nunique()!=1).sum()

In [ ]:
%%time
# some custom funcs used in agggregation
def mean_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.mean(np.diff(np.sort(x)))
    
def var_dwell_time(x):
    if len(x) == 1:
        return 0
    else:
        return np.var(np.diff(np.sort(x)))
    
def get_first(x):
    return x.iloc[0]

def get_last(x):
    return x.iloc[-1]

def n_clickouts(x):
    return (x=='clickout item').sum()

def click_rel_pos_avg(x):
    return np.mean(np.argwhere((x=='clickout item')))/len(x)

# define some aggs
session_aggs = {'timestamp': [np.ptp, mean_dwell_time, var_dwell_time],
                'step': ['max'],
                'action_type': ['nunique', n_clickouts, click_rel_pos_avg],
                'city': ['nunique', get_first],
                'platform': [get_first],
                'device': [get_first],
                'nfilters': ['mean', 'max', 'min', get_last],
                'nimps': ['max']
               }

In [24]:
session_grp = train.groupby('session_id')

In [ ]:
%%time
session_fts = session_grp.agg(session_aggs)
session_fts.columns = ['_'.join(col).strip() for col in session_fts.columns.values]

In [ ]:
# session_fts.columns.values
session_fts.head()

In [ ]:
# train[train.session_id=='62991f7c78f27']

### Create clickout features
---

In [20]:
import time

class Timer:    
    def __init__(self, task, profile):
        self.profile = profile
        self.task = task
        if self.task not in self.profile.keys():
            self.profile[self.task] = 0
        
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        self.profile[self.task] += np.round(self.interval, decimals=3)

In [21]:
# train[train.session_id=='62991f7c78f27']

In [67]:
meta.head()

,properties,ps,nprop,star,good_rating,satisfactory_rating,excellent_rating
item_id,,,,,,,
5121,terrace (hotel)|lift|childcare|from 3 stars|ra...,"[terrace (hotel), lift, childcare, from 3 star...",54,3,1,1,0
5555,terrace (hotel)|lift|from 3 stars|radio|from 4...,"[terrace (hotel), lift, from 3 stars, radio, f...",77,5,1,1,1
5684,terrace (hotel)|lift|from 3 stars|radio|4 star...,"[terrace (hotel), lift, from 3 stars, radio, 4...",62,4,1,1,0
5777,lift|childcare|from 3 stars|radio|4 star|from ...,"[lift, childcare, from 3 stars, radio, 4 star,...",69,4,1,1,0
6044,terrace (hotel)|childcare|from 3 stars|radio|b...,"[terrace (hotel), childcare, from 3 stars, rad...",45,3,1,1,0


In [66]:
meta.set_index('item_id', inplace=True)

In [74]:
meta.index

Int64Index([   5121,    5555,    5684,    5777,    6044,    6095,    6182,
               6257,    8022,    8087,
            ...
            1160238, 1161707, 1165463, 1169493, 1172094, 1174676, 1180610,
            1180626, 1188582, 1193336],
           dtype='int64', name='item_id', length=10000)

In [98]:
meta.index == 5121

array([ True, False, False, ..., False, False, False])

In [101]:
# 1) has the items appeared in previous impressions, and clickout
# session_grp
profile = {}
def compute_clicks(grp):
    with Timer('select_rows', profile) as t:
        last_row = grp.iloc[-1]
        above = grp.iloc[:-1]
    with Timer('exclude_nans', profile) as t:
        # get previous appeard impressions
        prev = above[above['impressions'].notnull()]
    with Timer('get_imp_list', profile) as t:
        prev_imps = prev['imps_list']
        unique_imps = [j for i in prev_imps for j in i]
    
    with Timer('get_price', profile) as t:
        imp_l = last_row['imps_list']
        prices = last_row['prices'].split('|')
        prices = [int(p) for p in prices]
        # whether the impression appeared before
        appeared = [int(i in unique_imps) for i in imp_l]
        # the location of the impression
        locs = list(range(len(imp_l)))
    
    with Timer('create_df', profile) as t:
    # build the df
        result = pd.DataFrame({'appeared': appeared, 'location': locs, 'price': prices}, index=imp_l)
        result.index.name = 'item_id'
    with Timer('rel_price_rank', profile) as t:
        result['rel_price_rank'] = result[['location', 'price']].sort_values(by='price')['location']/len(imp_l)
#     result['price_mean'] = (result['price'] - result['price'])/(len(imp_l)-1)
    with Timer('compute', profile) as t:
        result['price_mean'] = np.mean(result['price'])
        result['price_median'] = np.median(result['price'])
        result['diff_mean'] = result['price'] - result['price_mean']    
        result['diff_median'] = result['price'] - result['price_median']    
        result['diff_mean_rel'] = (result['price'] - result['price_mean'])/result['price']
        result['diff_median_rel'] = (result['price'] - result['price_median'])/result['price']
    with Timer('join', profile) as t:
        # fetch the meta data
        result = result.join(meta, on='item_id')
#         result = pd.merge(result, meta, left_index=True, right_index=True)
    
    with Timer('create_mean_meta', profile) as t:
        result['star_mean'] = result['star'].mean()
        result['gr_mean'] = result['good_rating'].mean()
        result['sr_mean'] = result['satisfactory_rating'].mean()
        result['er_mean'] = result['excellent_rating'].mean()
        result.reset_index(inplace=True)
        
    with Timer('create target', profile) as t:
        # get target
        ref = last_row['reference']
        result['target'] = (result['item_id'] == ref).astype(int)
    return result

n_runs = 5
n = 0
for k,v in session_grp:
    a = compute_clicks(v)
    n += 1
    if n == n_runs:
        break
profile_df = pd.Series(profile).rename_axis(index='task').reset_index(name='sec')
profile_df['per'] = (profile_df['sec']/profile_df['sec'].sum()*100).round(2)
profile_df
# 2) dwell time for each clickout
# 3) the rating and star and nproperty -> mean and median for 
#     the other impressions in list (or the quantile of these and prices)
# 4) location and relative location in the impressions list

/home/sam/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,task,sec,per
0,select_rows,0.001,1.56
1,exclude_nans,0.005,7.81
2,get_imp_list,0.000,0.00
3,get_price,0.000,0.00
4,create_df,0.005,7.81
5,rel_price_rank,0.011,17.19
6,compute,0.016,25.00
7,join,0.011,17.19
8,create_mean_meta,0.010,15.62
9,create target,0.005,7.81


,task,sec,per
0,select_rows,0.000,0.00
1,exclude_nans,0.005,8.47
2,get_imp_list,0.000,0.00
3,get_price,0.000,0.00
4,create_df,0.005,8.47
5,rel_price_rank,0.008,13.56
6,compute,0.015,25.42
7,join,0.021,35.59
8,create target,0.005,8.47


In [34]:
pprint(profile)

Pretty printing has been turned OFF


In [ ]:
%%time
dd = session_grp.apply(compute_clicks)

In [ ]:
%%time
# join on session features
dd = dd.reset_index(level=0).set_index('session_id')

In [ ]:
%%time
final = dd.join(session_fts)

### Questions:

1) timestamp to utc time, timezones? (this matters to e.g. hours of the timestamp)

2) does platform change within a session?

to do:

1) check the time range of train vs test